In [ ]:
import os
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms as T

# --- Cấu hình đường dẫn ---
ckpt_path = "/Users/braly/Desktop/lmvh/plant-identify/saved_models/vit_plants_full_checkpoint.ckpt"  # hoặc .ckpt file bạn có
image_path = "109_jpg.rf.40b08beaf6405ec5b5a1d708eb57022f.jpg"  # ảnh muốn test
root_dir = "/Users/braly/Desktop/lmvh/plant-identify/dataset/test"  # nếu muốn load classes từ dataset

# --- Device (hỗ trợ cuda / mps / cpu) ---
if torch.cuda.is_available():
    device = torch.device("cuda")
elif getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# --- Transforms (giữ như val_transform trong DataModule) ---
image_size = 224
val_transform = T.Compose([
    T.Resize((image_size, image_size)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

# --- Load tên lớp (nếu bạn dùng PlantDataset như khi train) ---
try:
    # Nếu bạn đã định nghĩa PlantDataset trong cùng file, dùng để lấy tên lớp
    dataset_for_classes = PlantDataset(root_dir=root_dir, split='train')
    classes = dataset_for_classes.classes
    print(f"Loaded {len(classes)} classes from dataset.")
except Exception as e:
    # fallback: nếu không thể load dataset, bạn có thể cung cấp thủ công:
    print("Không load được dataset để lấy classes:", e)
    # Ví dụ: classes = ["classA", "classB", ...]
    classes = None

# --- Load model từ checkpoint ---
model = None
try:
    # Thử load trực tiếp (Lightning lưu hparams vào checkpoint nên thường OK)
    model = ViTLightning.load_from_checkpoint(ckpt_path, map_location=device)
    print("Loaded model via ViTLightning.load_from_checkpoint()")
except Exception as e:
    print("load_from_checkpoint failed:", e)
    # Nếu thất bại, thử load state_dict và khởi tạo model thủ công.
    # Bạn có thể cần truyền num_classes nếu required.
    # Cố gắng lấy num_classes từ dataset nếu có
    if classes is not None:
        num_classes = len(classes)
    else:
        # nếu không có classes, hãy đặt đúng số lớp bạn đã train
        num_classes = 47  # <--- sửa theo số lớp thật nếu cần

    # Khởi tạo model (tham số phải phù hợp với lúc train)
    model = ViTLightning(num_classes=num_classes,
                         lr=3e-4,
                         weight_decay=1e-2,
                         backbone_name="vit_base_patch16_224",
                         pretrained=False,
                         freeze_backbone=False)
    # Load checkpoint file
    ckpt = torch.load(ckpt_path, map_location="cpu")
    if "state_dict" in ckpt:
        state_dict = ckpt["state_dict"]
    else:
        state_dict = ckpt  # có thể trực tiếp là state_dict
    # Một số key khi lưu bởi Lightning có tiền tố "model." hoặc "net."
    # Nếu keys mismatch, cố gắng strip tiền tố common (ví dụ "model.")
    new_state = {}
    for k, v in state_dict.items():
        new_k = k
        if k.startswith("model."):
            new_k = k[len("model."):]
        new_state[new_k] = v
    model.load_state_dict(new_state, strict=False)
    print("Loaded state_dict into newly created model.")

# --- Chuẩn bị model để inference ---
model.eval()
model.to(device)

# --- Load và tiền xử lý ảnh ---
img = Image.open(image_path).convert("RGB")
x = val_transform(img).unsqueeze(0).to(device)  # shape (1,C,H,W)

# --- Inference ---
with torch.no_grad():
    logits = model(x)  # (1, num_classes) — tùy backbone output trực tiếp logits
    if isinstance(logits, tuple) or isinstance(logits, list):
        logits = logits[0]
    probs = F.softmax(logits, dim=1)
    topk = torch.topk(probs, k=min(5, probs.shape[1]), dim=1)

top_probs = topk.values.cpu().numpy()[0]
top_idxs = topk.indices.cpu().numpy()[0]

# --- Hiển thị kết quả ---
if classes is None:
    # Nếu không có tên lớp, in idx
    for i, (idx, p) in enumerate(zip(top_idxs, top_probs), 1):
        print(f"#{i}: class_idx={idx}  prob={p:.4f}")
else:
    for i, (idx, p) in enumerate(zip(top_idxs, top_probs), 1):
        class_name = classes[int(idx)]
        print(f"#{i}: {class_name} (idx={idx})  prob={p:.4f}")

# (Tùy ý) Hiển thị ảnh trong notebook (nếu dùng notebook)
try:
    from IPython.display import display
    print("Input image:")
    display(img)
except Exception:
    pass


SyntaxError: illegal target for annotation (2282645995.py, line 36)